In [89]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from multielo import MultiElo, Player, Tracker
from mktools.get_data import load_data_pd
from alive_progress import alive_it
from mktools.validate_data import validate_bad_uids

load_dotenv()

True

In [90]:
df = load_data_pd(
    sheet_name="data",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

invalid, valid = validate_bad_uids(df=df, return_valid=True)

In [93]:
valid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
20615,6289,735,Cooper,D.K.,Bowser's Castle,2,3,2024-07-15,11
20616,6289,735,Konnor,Yoshi,Bowser's Castle,3,3,2024-07-15,11
20617,6290,736,Cooper,Toad,Yoshi Valley,1,3,2024-07-16,11
20618,6290,736,Cole,Peach,Yoshi Valley,2,3,2024-07-16,11


In [94]:
rated_seasons = []

for season in alive_it(
    it=valid["SEASON"].unique(),
    total=valid["SEASON"].unique().shape[0],
    theme="classic",
):

    tdf = (
        valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == season)]
        .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
        .sort_values(by=["UID", "PLACE"])
        .copy()
        .reset_index(drop=True)
    )

    pdf = (
        tdf.pivot(index="UID", columns="PLACE", values="NAME")
        .reset_index()
        .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
        .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
        .groupby("UID")
        .first()
        .set_index("DATE")
        .reset_index()
        .rename(columns={"DATE": "date"})
    )

    exp_elo = MultiElo(score_function_base=1.5)
    tracker = Tracker(elo_rater=exp_elo)
    tracker.process_data(pdf)

    rdf = tracker.get_current_ratings()

    rdf["SEASON"] = season

    rated_seasons.append(rdf)

[!                                       ] (!) 0/12 [0%] in 0.0s (0.00/s) 


ValueError: Index contains duplicate entries, cannot reshape

In [95]:
rated_seasons[-1]

IndexError: list index out of range

In [64]:
pd.concat(rated_seasons).sort_values(by=["SEASON", "rank"]).reset_index(drop=True)

,rank,player_id,n_games,rating,SEASON
0,1,Cooper,142,1431.138670,0
1,2,Regan,58,1212.174791,0
2,3,Cole,33,1153.182061,0
3,4,Matt,68,1138.867777,0
4,5,Blake,29,1085.996876,0
...,...,...,...,...,...
235,16,Randy,10,876.515824,11
236,17,Colton,21,872.667841,11
237,18,Domingo,26,840.244082,11
238,19,Antonio,15,839.396319,11


In [66]:
tdf

,UID,NAME,PLACE,DATE
0,1,Cooper,1,2021-09-20
1,1,Connor,2,2021-09-20
2,1,Triston,3,2021-09-20
3,1,Cole,4,2021-09-20
4,2,Cooper,1,2021-09-20
...,...,...,...,...
21641,6288,Regan,3,2024-07-15
21642,6288,Konnor,4,2024-07-15
21643,6289,Regan,1,2024-07-15
21644,6289,Cooper,2,2024-07-15


In [67]:
df

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
21641,6288,735,Regan,Yoshi,Moo Moo Farm,3,4,2024-07-15,11
21642,6288,735,Konnor,Bowser,Moo Moo Farm,4,4,2024-07-15,11
21643,6289,735,Regan,Toad,Bowser's Castle,1,3,2024-07-15,11
21644,6289,735,Cooper,D.K.,Bowser's Castle,2,3,2024-07-15,11


In [96]:
msk = (valid["UID"].value_counts().reset_index()["count"] > 4 ) | (valid["UID"].value_counts().reset_index()["count"] < 2)

valid[valid["UID"].isin(valid["UID"].value_counts().index[msk])]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [110]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 0)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

tdf

,UID,NAME,PLACE,DATE
0,1,Cooper,1,2021-09-20
1,1,Connor,2,2021-09-20
2,1,Triston,3,2021-09-20
3,1,Cole,4,2021-09-20
4,2,Cooper,1,2021-09-20
...,...,...,...,...
1559,689,Cole,4,2021-11-10
1560,692,Matt,1,2021-11-10
1561,692,Cooper,2,2021-11-10
1562,692,Blake,3,2021-11-10


In [111]:
tdf["UID"].value_counts()

UID
1      4
390    4
432    4
424    4
423    4
      ..
197    4
193    4
192    4
191    4
692    4
Name: count, Length: 391, dtype: int64

In [107]:
pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

pdf

,date,1st,2nd,3rd,4th
0,2022-01-25,Cooper,Blake,Connor,Jake
1,2022-01-25,Cooper,Connor,Jake,Blake
2,2022-01-25,Cooper,Blake,Jake,Connor
3,2022-01-25,Connor,Cole,Blake,Robert
4,2022-01-27,Luke,Matt,Chandler,Robert
...,...,...,...,...,...
230,2022-03-12,Cole,Blake,Connor,Jake
231,2022-03-13,Cooper,Regan,Cole,Triston
232,2022-03-13,Cooper,Cole,Triston,Regan
233,2022-03-13,Cooper,Regan,Cole,Triston


In [101]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 0)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=1.5)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

ValueError: Index contains duplicate entries, cannot reshape

In [97]:
tdf = (
    valid.drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=1.5)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# import altair as alt

# df = tracker.get_history_df()
# alt.Chart(df).mark_line(point=True).encode(
#     x="date:T",
#     y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
#     color="player_id:N",
# )